# The Keras Way

In [71]:
import pandas as pd
from keras import Sequential
from keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

In [16]:
df = pd.read_csv("/Users/jleem/dev/personal-website/assets/data/hockey/final-cleaned-stats.csv")

In [27]:
df.index = df['player_id']
df.drop("player_id",axis=1)

,games_played,timeOnIce,goals,assists,shots,hits,powerPlayGoals,powerPlayAssists,penaltyMinutes,faceOffWins,faceoffTaken,takeaways,giveaways,shortHandedGoals,shortHandedAssists,blocked,plusMinus,evenTimeOnIce,shortHandedTimeOnIce,powerPlayPoints,shortHandedPoints,faceOffPercentage
player_id,,,,,,,,,,,,,,,,,,,,,,
8446485,47,626.978723,6,5,57,45,0,0,12,95,168,16,8,0,0,11,1,573.255319,51.191489,0,0,56.547619
8448208,460,1039.756522,120,202,1041,114,36,48,260,2,8,199,276,0,0,73,47,872.436957,1.308696,84,0,25.000000
8450725,81,966.283951,14,34,132,81,6,11,35,52,97,16,32,0,1,25,13,761.962963,43.308642,17,1,53.608247
8456283,47,1075.510638,0,8,23,55,0,0,33,0,0,4,15,0,0,67,-9,896.234043,170.702128,0,0,0.000000
8457063,152,1416.460526,27,69,323,90,11,45,48,0,0,60,56,0,1,171,19,1049.921053,133.348684,56,1,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8480839,82,1269.329268,9,35,177,97,5,15,34,0,0,38,79,0,0,66,-13,1088.743902,9.500000,20,0,0.000000
8480943,53,840.905660,3,4,43,57,0,0,14,0,0,11,34,0,0,48,0,835.075472,4.113208,0,0,0.000000
8480944,65,680.800000,1,12,65,58,0,0,18,204,407,30,12,0,0,55,5,581.415385,95.230769,0,0,50.122850


In [28]:
pinfo = pd.read_csv("/Users/jleem/dev/personal-website/assets/data/hockey/player_info.csv")

In [29]:
pinfo

,player_id,firstName,lastName,nationality,birthCity,primaryPosition,birthDate,link
0,8467412,Alexei,Ponikarovsky,UKR,Kiev,LW,1980-04-09,/api/v1/people/8467412
1,8468501,Anton,Volchenkov,RUS,Moscow,D,1982-02-25,/api/v1/people/8468501
2,8459670,Kimmo,Timonen,FIN,Kuopio,D,1975-03-18,/api/v1/people/8459670
3,8471233,Travis,Zajac,CAN,Winnipeg,C,1985-05-13,/api/v1/people/8471233
4,8455710,Martin,Brodeur,CAN,Montreal,G,1972-05-06,/api/v1/people/8455710
...,...,...,...,...,...,...,...,...
2404,8478040,Austin,Poganski,USA,St. Cloud,RW,1996-02-16,/api/v1/people/8478040
2405,8480011,Klim,Kostin,RUS,Penza,C,1999-05-05,/api/v1/people/8480011
2406,8480384,Alexander,True,DNK,Kobenhavn,C,1997-07-17,/api/v1/people/8480384
2407,8480021,Jack,Studnicka,CAN,Windsor,C,1999-02-18,/api/v1/people/8480021


In [89]:
df['position'] = [ pinfo['player_id'] == _ for _ in df.index ]


In [90]:
train, test = train_test_split(df, test_size = 0.25)

In [91]:
model = Sequential()
model.add(Dense(64, input_shape = (train.shape[1],), activation = 'relu'))
model.add(Dense(128, activation = 'sigmoid'))
model.add(Dropout(0.3))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(4, activation = "softmax"))

In [92]:
# Convert string to integer, then use keras' one-hot encoding
label_ids = dict([ (v,i) for i,v in enumerate(set(train_y)) ])

In [65]:
train_y_encoded = to_categorical([label_ids[_] for _ in train['position']], num_classes=4)
test_y_encoded = to_categorical([label_ids[_] for _ in test_y], num_classes=4)

In [74]:
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [76]:
model.fit(train.values, train_y_encoded, epochs=5, batch_size=32)

Epoch 1/5
1025/1025 [==============================] - 0s 112us/step - loss: 1.3577 - accuracy: 0.3288
Epoch 2/5
1025/1025 [==============================] - 0s 98us/step - loss: 1.3581 - accuracy: 0.3327
Epoch 3/5
1025/1025 [==============================] - 0s 97us/step - loss: 1.3601 - accuracy: 0.3298
Epoch 4/5
1025/1025 [==============================] - 0s 99us/step - loss: 1.3621 - accuracy: 0.3366
Epoch 5/5
1025/1025 [==============================] - 0s 97us/step - loss: 1.3563 - accuracy: 0.3395


In [81]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(train.values, train_y)

/Users/jleem/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [82]:
rf.predict(train.values)

array(['LW', 'D', 'D', ..., 'C', 'D', 'C'], dtype=object)

In [83]:
correct, total = 0,0
for a,b in zip(rf.predict(train.values), train_y):
    if a==b:
        correct += 1
    total += 1

print(correct*1.0 / total)

0.9834146341463414


# The PyTorch Way

In [2]:
import torch.nn as nn